In [1]:
%pip install datasets transformers
%pip install bert_score
%pip install evaluate
%pip install sentence-transformers
%pip install accelerate -U
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 471.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [8]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
from datasets import load_dataset
from torchtext.datasets import multi30k, Multi30k
from torch import Tensor
import torch
import evaluate as E
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.nn import Transformer
import math
from torch.nn.utils.rnn import pad_sequence
from timeit import default_timer as timer
from torch.utils.data import DataLoader
import numpy as np
import math
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import T5Tokenizer, T5Model
import pandas as pd
from transformers import pipeline
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, BertConfig
from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sentence_transformers import evaluation
from transformers import AutoTokenizer
from scipy.stats import pearsonr
from torch.utils.data import DataLoader, TensorDataset


# Loading 1 dataset

In [6]:
path = '/content/drive/MyDrive/A3_task1_data_files'

task1_train_data =  pd.read_csv(path + '/train.csv', sep = "\t")
task1_train_data.dropna(inplace=True)
task1_train_data.reset_index(drop=True, inplace=True)

path = '/content/drive/MyDrive/A3_task1_data_files'
task1_val_data =pd.read_csv(path +'/dev.csv', sep = "\t")
task1_val_data = task1_val_data.rename(columns = {'setence1':'sentence1'})
task1_val_data.dropna(inplace=True)
task1_val_data.reset_index(drop=True, inplace=True)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)


cuda


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1A

In [9]:
newbert = BertForSequenceClassification.from_pretrained("bert-base-uncased")
newbert.classifier = nn.Linear(newbert.config.hidden_size, 1)
newbert = newbert.to(DEVICE)

path = "/content/drive/MyDrive/"
newbert.load_state_dict(torch.load(path + "model1A.pt"))
all_val_predictions = []
all_val_targets = []

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, model_name):
        self.data = dataframe
        self.tokenizer = BertTokenizer.from_pretrained(model_name)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence1 = str(self.data.iloc[idx]['sentence1'])
        sentence2 = str(self.data.iloc[idx]['sentence2'])
        label = self.data.iloc[idx]['score']
        inputs = self.tokenizer(sentence1, sentence2, return_tensors="pt", max_length = 512, padding="max_length", truncation=True, add_special_tokens = True)
        return inputs, torch.tensor(label, dtype=torch.long)

    def get_encoded_data(self):
        input_ids = []
        attention_masks = []
        labels = []
        for i in range(self.__len__()):
            input, label = self.__getitem__(i)
            input_ids.append(input['input_ids'])
            attention_masks.append(input['attention_mask'])
            labels.append(label.float()/5.0)

        input_ids = torch.cat(input_ids, dim=0).squeeze(0)
        attention_masks = torch.cat(attention_masks, dim=0).squeeze(0)
        labels = np.array(labels)
        labels = labels.reshape(-1, 1)
        labels = torch.tensor(labels)
        return input_ids, attention_masks, labels

    def get_dataloader(self, batch_size):
        input_ids, attention_masks, labels = self.get_encoded_data()
        dataset = TensorDataset(input_ids, attention_masks, labels)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return dataloader

task1_val_dataset = CustomDataset(task1_val_data,"bert-base-uncased")
task1_train_dataloader = task1_val_dataset.get_dataloader(16)
task1_val_dataloader = task1_val_dataset.get_dataloader(16)

with torch.no_grad():
  for val_data in task1_val_dataloader:
    val_inputs,val_attention_mask,val_labels=val_data
    val_inputs = val_inputs.to(DEVICE)
    val_attention_mask = val_attention_mask.to(DEVICE)
    val_labels = val_labels.to(DEVICE)
    val_outputs = newbert(input_ids=val_inputs, attention_mask=val_attention_mask, labels=val_labels)

    all_val_predictions.extend(val_outputs["logits"].view(-1).cpu().numpy())
    all_val_targets.extend(val_labels.view(-1).cpu().numpy())

pearson_corr, _ = pearsonr(all_val_targets*5, all_val_predictions*5)
print("Pearson Correlation Coefficient on Val Set:", pearson_corr)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Pearson Correlation Coefficient on Dev Set: 0.7958015662323207


## 1B

In [10]:
def metrics_1(data, model):
    values = []
    for i in range(len(data)):
        embed1 = model.encode(data.iloc[i]['sentence1'])
        embed2 = model.encode(data.iloc[i]['sentence2'])
        similarity = np.dot(embed1, embed2)/(np.linalg.norm(embed1)*np.linalg.norm(embed2))
        values.append(similarity*5)
    pearson = np.corrcoef(values, data['score'])
    print("Pearson Correlation: ", pearson[0][1])

In [11]:
model_1B = SentenceTransformer('distilbert-base-nli-mean-tokens')
model_1B = model_1B.to(DEVICE)
print("Val Metrics")
metrics_1(task1_val_data, model_1B)
# Pearson Correlation:  0.7923100957816998

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Val Metrics
Pearson Correlation:  0.7923100957816998


## 1C

In [13]:
path = "/content/drive/MyDrive/"
# path = ""
model_1C = SentenceTransformer(path + 'model_1C')
model_1C = model_1C.to(DEVICE)

# Pearson Correlation:  0.8572838200887052


In [ ]:
print("Val Metrics")
metrics_1(task1_val_data, model_1C)

# 1C Inference Pipeline

In [14]:
path = "/content/drive/MyDrive/"

# path = ""

test_csv = pd.read_csv(path +"A3_task1_data_files/sample_test.csv", sep='\t')
test_csv.head()

scores = []
for i in range(len(test_csv)):
    # print("Calculating for: ", i)
    embed1 = model_1C.encode(test_csv.iloc[i]['setence1'])
    embed2 = model_1C.encode(test_csv.iloc[i]['sentence2'])
    similarity = np.dot(embed1, embed2)/(np.linalg.norm(embed1)*np.linalg.norm(embed2))
    scores.append(similarity*5)


demo_csv = pd.DataFrame({
    'id': test_csv['id'],
    'score': scores,
    'setence1': test_csv['setence1'],
    'sentence2': test_csv['sentence2']})


demo_csv.to_csv('new_dataframe.csv', sep='\t', index=False)


# Loading 2 dataset

In [15]:
task2_train_data = load_dataset("wmt16", "de-en", split="train[:50000]")
task2_val_data = load_dataset("wmt16", "de-en", split="validation")
task2_test_data = load_dataset("wmt16", "de-en", split="test")


Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [16]:
bleu = E.load("bleu")
meteor_metric = E.load("meteor")
bertscore_metric = E.load("bertscore")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


## 2A

In [17]:
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'
token_transform = {}
vocab_transform = {}

token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    for data_sample in data_iter:
        yield token_transform[language](data_sample['translation'][language])


UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    train_iter = task2_train_data
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln],
                                               vocab_transform[ln],
                                               tensor_transform)

def collate_fn(batch):
    src_batch, tgt_batch = [], []

    for pair in batch:
        src_sample = pair['translation']['de']
        tgt_sample = pair['translation']['en']
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

def decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


def translate_2a(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

def test_metrics_2a(model, data):
  model.eval()
  references = []
  predictions = []
  for id, pair in enumerate(data):

      src_sample = pair['translation']['de']
      tgt_sample = pair['translation']['en']

      predictions.append(translate_2a(model, src_sample))
      references.append([tgt_sample])

  bleu_score = bleu.compute(predictions=predictions, references=references)
  meteor_score = meteor_metric.compute(predictions=predictions, references=references)
  bertscore_score = bertscore_metric.compute(predictions=predictions, references=references, lang="en")

  bs_pre = np.mean(np.array(bertscore_score['precision']))
  bs_recall = np.mean(np.array(bertscore_score['recall']))
  bs_f1 = np.mean(np.array(bertscore_score['f1']))

  print(f'BLEU Score: {bleu_score}')
  print(f'METEOR Score: {meteor_score}')
  print(f'BERTScore-precision: {bs_pre}')
  print(f'BERTScore-recall: {bs_recall}')
  print(f'BERTScore-f1: {bs_f1}')

In [18]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 64
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
path = "/content/drive/MyDrive/"

model_2A = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
model_2A.load_state_dict(torch.load(path + 'model_2A.pt'))
model_2A = model_2A.to(DEVICE)



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
print("Val Metrics")
test_metrics_2a(model_2A, task2_val_data)
print("Test Metrics")
test_metrics_2a(model_2A, task2_test_data)

## 2B

In [19]:
model_name = "t5-small"
model_2B = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer_2b = T5Tokenizer.from_pretrained(model_name)
model_2B = model_2B.to(DEVICE)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
def translate_2b(sentence):
    prefix = "translate English to German: "
    input_text = prefix + sentence
    input_ids = tokenizer_2b.encode(input_text, return_tensors="pt")
    input_ids = input_ids.to(DEVICE)
    output_ids = model_2B.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
    translated_sentence = tokenizer_2b.decode(output_ids[0], skip_special_tokens=True)
    return translated_sentence

def test_metrics_2b(data):
  references = []
  predictions = []
  for id, pair in enumerate(data):
      # print(id,"/", len(data))

      tgt_sample = pair['translation']['de']
      src_sample = pair['translation']['en']

      predictions.append(translate_2b(src_sample))
      references.append([tgt_sample])

  bleu_score = bleu.compute(predictions=predictions, references=references)
  meteor_score = meteor_metric.compute(predictions=predictions, references=references)
  bertscore_score = bertscore_metric.compute(predictions=predictions, references=references, lang="de")

  bs_pre = np.mean(np.array(bertscore_score['precision']))
  bs_recall = np.mean(np.array(bertscore_score['recall']))
  bs_f1 = np.mean(np.array(bertscore_score['f1']))


  print(f'BLEU Score: {bleu_score}')
  print(f'METEOR Score: {meteor_score}')
  print(f'BERTScore-precision: {bs_pre}')
  print(f'BERTScore-recall: {bs_recall}')
  print(f'BERTScore-f1: {bs_f1}')

In [ ]:
model_2B.eval()
print("2B-Val Metrics")
test_metrics_2b(task2_val_data)
print("2B-Test Metrics")
test_metrics_2b(task2_test_data)

2B-Val Metrics


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BLEU Score: {'bleu': 0.2823125666465826, 'precisions': [0.5945855614973262, 0.3374072253049566, 0.21694413614502406, 0.14595002996118073], 'brevity_penalty': 1.0, 'length_ratio': 1.0140081337550837, 'translation_length': 44880, 'reference_length': 44260}
METEOR Score: {'meteor': 0.5515488896080301}
BERTScore-precision: 0.8656165648188224
BERTScore-recall: 0.8643085958771906
BERTScore-f1: 0.864771080709592
2B-Test Metrics
BLEU Score: {'bleu': 0.3268254810286919, 'precisions': [0.6340116232529098, 0.38745753590528403, 0.26043603775924057, 0.18072829969887969], 'brevity_penalty': 0.9966754605913349, 'length_ratio': 0.9966809746445611, 'translation_length': 62461, 'reference_length': 62669}
METEOR Score: {'meteor': 0.5902469429945494}
BERTScore-precision: 0.8778189018632062
BERTScore-recall: 0.8741395657521241
BERTScore-f1: 0.875798528374891


## 2C

In [24]:
path = ""
path = "/content/drive/MyDrive/"
model_2C = T5ForConditionalGeneration.from_pretrained(path + "model_2C")
checkpoint = "google-t5/t5-small"
tokenizer_2c = AutoTokenizer.from_pretrained(checkpoint)
model_2C = model_2C.to(DEVICE)
model_2C.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [22]:
def translate_2c(sentence):
    prefix = "translate German to English: "
    input_text = prefix + sentence
    input_ids = tokenizer_2c.encode(input_text, return_tensors="pt")
    input_ids = input_ids.to(DEVICE)
    output_ids = model_2C.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
    translated_sentence = tokenizer_2c.decode(output_ids[0], skip_special_tokens=True)
    return translated_sentence

def test_metrics_2c(data):
  references = []
  predictions = []
  for id, pair in enumerate(data):
    #   print(id,"/", len(data))

      tgt_sample = pair['translation']['en']
      src_sample = pair['translation']['de']

      predictions.append(translate_2c(src_sample))
      references.append([tgt_sample])

  bleu_score = bleu.compute(predictions=predictions, references=references)
  meteor_score = meteor_metric.compute(predictions=predictions, references=references)
  bertscore_score = bertscore_metric.compute(predictions=predictions, references=references, lang="en")

  bs_pre = np.mean(np.array(bertscore_score['precision']))
  bs_recall = np.mean(np.array(bertscore_score['recall']))
  bs_f1 = np.mean(np.array(bertscore_score['f1']))

  print(f'BLEU Score: {bleu_score}')
  print(f'METEOR Score: {meteor_score}')
  print(f'BERTScore-precision: {bs_pre}')
  print(f'BERTScore-recall: {bs_recall}')
  print(f'BERTScore-f1: {bs_f1}')

In [ ]:

print("2C-Val Metrics")
test_metrics_2c(task2_val_data)
print("2C-Test Metrics")
test_metrics_2c(task2_test_data)

2C-Val Metrics


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BLEU Score: {'bleu': 0.16651898208894583, 'precisions': [0.5077333215827973, 0.22288808163076423, 0.11548486103329841, 0.06456340635606295], 'brevity_penalty': 0.9770240322294348, 'length_ratio': 0.9772839825162026, 'translation_length': 45388, 'reference_length': 46443}
METEOR Score: {'meteor': 0.44203751154922416}
BERTScore-precision: 0.8985999935010767
BERTScore-recall: 0.9083440595093055
BERTScore-f1: 0.9032909002871269
2C-Test Metrics


KeyboardInterrupt: 

## 2-Inference Pipeline

In [35]:
german_sentence = "Ich  bin Nalish" #Enter input here
print("2A")
print("Translated sentence is :", translate_2a(model_2A, german_sentence))

english_sentence = "How are you?" #Enter Input here
print("2B")
print("Translated sentence is :", translate_2b(english_sentence))

print("2C")
print("Translated sentence is :", translate_2c(german_sentence))


2A
Translated sentence is :  I am in favour of the TSE ban . 
2B
Translated sentence is : Wie sind Sie?
2C
Translated sentence is : I bin Nalish


# Input CSV

In [27]:
# german_sentences = ["Hallo", "Wie geht es Ihnen?"]
# english_sentences = ["Hello", "How are you?"]

# # Create a pandas DataFrame
# data = {'de': german_sentences, 'en': english_sentences}
# df = pd.DataFrame(data)

# print(df)

                   de            en
0               Hallo         Hello
1  Wie geht es Ihnen?  How are you?


In [33]:
path = ""
german_csv = pd.read_csv(path)
# german_csv = df
german_sentences = german_csv['de']

english_sentences_2a = []
for sentence in german_sentences:

    english_sentences_2a.append(translate_2a(model_2A, sentence))

de_en_2a = pd.DataFrame({
    'de': german_csv['de'],
    'en': english_sentences_2a})
de_en_2a.to_csv('de_en_2a.csv', index=False)


english_sentences_2c = []
for sentence in german_sentences:
    english_sentences_2c.append(translate_2c(sentence))

de_en_2c = pd.DataFrame({
    'de': german_csv['de'],
    'en': english_sentences_2c})
de_en_2c.to_csv('de_en_2c.csv', index=False)